<a href="https://colab.research.google.com/github/WISEPLAT/_NN/blob/main/1_%D1%80%D0%B0%D1%81%D0%BF%D0%BE%D0%B7%D0%BD%D0%B0%D0%B5%D0%BC_%D1%80%D1%83%D0%BA%D0%BE%D0%BF%D0%B8%D1%81%D0%BD%D1%8B%D0%B5_%D1%86%D0%B8%D1%84%D1%80%D1%8B_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.datasets import mnist # subroutines for fetching the MNIST dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
import numpy as np

In [ ]:
batch_size = 128 # in each iteration, we consider 128 training examples at once
num_epochs = 20 # we iterate twenty times over the entire training set
hidden_size = 512 # there will be 512 neurons in both hidden layers

In [ ]:
num_train = 60000 # there are 60000 training examples in MNIST
num_test = 10000 # there are 10000 test examples in MNIST

height, width, depth = 28, 28, 1 # MNIST images are 28x28 and greyscale
num_classes = 10 # there are 10 classes (1 per digit)

(X_train, y_train), (X_test, y_test) = mnist.load_data() # fetch MNIST data

# Reshape - DON'T NEED
#X_train = X_train.reshape(num_train, height * width) # Flatten data to 1D
#X_test = X_test.reshape(num_test, height * width) # Flatten data to 1D
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')

X_train /= 255 # Normalise data to [0, 1] range
X_test /= 255 # Normalise data to [0, 1] range

#X_train /= np.max(X_train) # Normalise data to [0, 1] range
#X_test /= np.max(X_train) # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

In [ ]:
X_train.shape, Y_train.shape

((60000, 28, 28), (60000, 10))

In [ ]:
X_train = X_train[:,:,:,np.newaxis]
X_train.shape

(60000, 28, 28, 1)

In [ ]:
X_test.shape, Y_test.shape

((10000, 28, 28), (10000, 10))

In [ ]:
X_test = X_test[:,:,:,np.newaxis]
X_test.shape

(10000, 28, 28, 1)

In [ ]:
#MLP
# inp = Input(shape=(height * width,)) # Our input is a 1D vector of size 784
# hidden_1 = Dense(hidden_size, activation='relu')(inp) # First hidden ReLU layer
# hidden_2 = Dense(hidden_size, activation='relu')(hidden_1) # Second hidden ReLU layer
# out = Dense(num_classes, activation='softmax')(hidden_2) # Output softmax layer

# model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
#model.summary()

In [ ]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

In [ ]:
#CNN

inp = Input(shape=(height, width, depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

In [ ]:
model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_33 (Conv2D)          (None, 10, 10, 32)        320       
                                                                 
 conv2d_34 (Conv2D)          (None, 4, 4, 32)          9248      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 2, 2, 32)         0         
 g2D)                                                            
                                                                 
 dropout_24 (Dropout)        (None, 2, 2, 32)          0         
                                                                 
 conv2d_35 (Conv2D)          (None, 1, 1, 64)          18496     
                                                           

In [ ]:
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

In [ ]:
model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

Epoch 1/200
1688/1688 [==============================] - 16s 9ms/step - loss: 0.5120 - accuracy: 0.8364 - val_loss: 0.1274 - val_accuracy: 0.9608
Epoch 2/200
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2201 - accuracy: 0.9362 - val_loss: 0.0863 - val_accuracy: 0.9743
Epoch 3/200
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1763 - accuracy: 0.9504 - val_loss: 0.0672 - val_accuracy: 0.9805
Epoch 4/200
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1548 - accuracy: 0.9547 - val_loss: 0.0623 - val_accuracy: 0.9807
Epoch 5/200
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1378 - accuracy: 0.9598 - val_loss: 0.0611 - val_accuracy: 0.9822
Epoch 6/200
1688/1688 [==============================] - 16s 9ms/step - loss: 0.1259 - accuracy: 0.9636 - val_loss: 0.0578 - val_accuracy: 0.9830
Epoch 7/200
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1166 - accuracy: 0.9668 - val_loss: 0.0587 -

In [ ]:
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

313/313 [==============================] - 1s 3ms/step - loss: 0.0886 - accuracy: 0.9847


[0.08859189599752426, 0.9847000241279602]